# GW Releases

In [1]:
import requests
import pandas as pd
import re
import json
from IPython.core.display import display, HTML
from modules.Network import *
display(HTML("<style>.container { width:70% !important; }</style>"))

In [2]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

We get the json file with the information from https://wmwaredata.s3.us-east-2.amazonaws.com/gw_releases.json

In [3]:
url = 'https://wmwaredata.s3.us-east-2.amazonaws.com/gw_releases.json'
r = requests.get(url, allow_redirects=True)
open('data/gw_releases.json', 'wb').write(r.content)

6145

In [4]:
df = pd.read_json('data/gw_releases.json')

In [5]:
df

,commit_url,hash,hash_short,release_date,release_name,release_tag,repo_name,repo_url,sub_commit_url,sub_hash,sub_hash_short,sub_name,sub_repo_commit_url
0,https://github.com/k8-proxy/GW-proxy/tree/9024...,902403ed9f9834e1b393dd05d308e514c536981b,902403e,2021-01-08T16:40:48Z,,v0.5.1,GW-proxy,https://github.com/k8-proxy/GW-proxy,None,None,None,None,None
1,https://github.com/k8-proxy/s-k8-proxy-rebuild...,ffbcabeaba991606b3e86d9f44f08fe4e2f4a26d,ffbcabe,2021-01-20T09:34:25Z,,v0.1.1,s-k8-proxy-rebuild,https://github.com/k8-proxy/s-k8-proxy-rebuild,None,None,None,None,None
2,https://github.com/k8-proxy/aws-jmeter-test-en...,47dd00c402359a072c2fe614b1be6eb86d2fe06a,47dd00c,2021-01-29T12:29:04Z,- Instructions simplified\r\n- Bugs fixed\r\n-...,v0.2.1,aws-jmeter-test-engine,https://github.com/k8-proxy/aws-jmeter-test-en...,None,None,None,None,None
3,https://github.com/k8-proxy/icap-infrastructur...,9027b550a263af153220a3a98377b8e34cf4e7de,9027b55,2021-02-15T12:18:08Z,,v1.0.1,icap-infrastructure,https://github.com/k8-proxy/icap-infrastructure,None,None,None,None,None
4,https://github.com/k8-proxy/k8-rebuild/tree/ce...,ce00c47c470529d47061ed25f4faf064e416b470,ce00c47,2021-02-15T11:03:05Z,,v0.1.2,k8-rebuild,https://github.com/k8-proxy/k8-rebuild,https://github.com/k8-proxy/k8-rebuild-file-dr...,79b14699988edbe0fb0c64a3b2c4d4c0eb60587e,79b1469,k8-rebuild-file-drop,https://github.com/k8-proxy/k8-rebuild-file-drop
5,https://github.com/k8-proxy/k8-rebuild/tree/ce...,ce00c47c470529d47061ed25f4faf064e416b470,ce00c47,2021-02-15T11:03:05Z,,v0.1.2,k8-rebuild,https://github.com/k8-proxy/k8-rebuild,https://github.com/k8-proxy/k8-rebuild-rest-ap...,34a3f777159ff566102850064d2017a6054b7591,34a3f77,k8-rebuild-rest-api,https://github.com/k8-proxy/k8-rebuild-rest-api
6,https://github.com/k8-proxy/k8-rebuild-folder-...,80db47c82f60311d49140253dd5418d874cb85a5,80db47c,2021-02-03T14:49:50Z,Fixes on v0.1.1:\r\n1.- Some zip files were no...,v0.1.1,k8-rebuild-folder-to-folder,https://github.com/k8-proxy/k8-rebuild-folder-...,None,None,None,None,None
7,https://github.com/k8-proxy/vmware-scripts/tre...,0d97cbe34d1464acb75fd56a0973c65cca5f282a,0d97cbe,2021-02-15T11:05:13Z,,v0.1.6,vmware-scripts,https://github.com/k8-proxy/vmware-scripts,None,None,None,None,None


In [6]:
# CREATE JSON FILE WITH NODES AND EDGES

network = Network()
network.groups = ['GW-Releases', 'Repo', 'Sub_Repo']
network.groupSettings = group_settings_GW

# GW-Releases node
network.add_node(label='GW-Releases', group='GW-Releases')

for repo in set(list(df['repo_url'])):
    # REPO
    repo_name = re.sub('https://github.com/', '', repo)
    #hash_ = df[df['repo_url']==repo]['hash'].iloc[0]
    hash_short = df[df['repo_url']==repo]['hash_short'].iloc[0]
    tag = df[df['repo_url']==repo]['release_tag'].iloc[0]
    label = repo_name + '\n' + tag + '\n' + hash_short
    network.add_node(label, group='Repo')
    network.add_edge(network.nodesIds['GW-Releases'][0], network.nodesIds[label][0])
    
    # SUB REPO
    df1 = df[df['repo_url'] == repo].dropna().reset_index()
    for i in range(len(df1)):
        sub_repo = df1['sub_repo_commit_url'].iloc[i]
        sub_repo_name = re.sub('https://github.com/', '', sub_repo)
        #sub_hash = df1[df1['sub_repo_commit_url']==sub_repo]['sub_hash'].iloc[0]
        sub_hash_short = df1[df1['sub_repo_commit_url']==sub_repo]['sub_hash_short'].iloc[0]
        label2 = sub_repo_name + '\n' + sub_hash_short
        network.add_node(label2, group='Sub_Repo')
        network.add_edge(network.nodesIds[label][0], network.nodesIds[label2][0])
        
network.save_to_json('data/gw_releases_nodes_edges.json')

In [7]:
%%html
<div id="mynetwork0"></div>

In [8]:
%%javascript
requirejs.config({
    paths: {
        vis: 'vis'
    }
});


require(['vis'], function(vis){
    
    var json = $.getJSON("data/gw_releases_nodes_edges.json")
      .done(function(data){
        var data = {
          nodes: data.nodes,
          edges: data.edges
        };
        var network = new vis.Network(container, data, options);
      });
    
    var options = {
      width: '1200px',
      height: '900px',
      locale: 'en',
      physics: false,
      interaction: {
        hover:true, 
        tooltipDelay: 300
      },        
      layout: {
        randomSeed: 1,
        improvedLayout: true,
        hierarchical: {
          enabled: true,
          direction: 'LR',
          nodeSpacing: 200,
          levelSeparation: 250,  
          sortMethod: 'directed',
        },
      },
    }; 
    
    var container = document.getElementById("mynetwork0");
    
});

<IPython.core.display.Javascript object>

In [9]:
%%html
<div id="mynetwork1"></div>

In [10]:
%%javascript
requirejs.config({
    paths: {
        vis: 'vis'
    }
});

require(['vis'], function(vis){ 
    
    var json = $.getJSON("data/gw_releases_nodes_edges.json")
      .done(function(data){
        var data = {
          nodes: data.nodes,
          edges: data.edges
        };
        var network = new vis.Network(container, data, options);
      });
       
    var options = {      
      width: '1200px',
      height: '1000px',
      physics: true,
    };    
    
    var container = document.getElementById("mynetwork1");
});

<IPython.core.display.Javascript object>